# Agent build with langchain, memory, react_agent

#### Basic Setup of Environment and Configuration

In [31]:
# Basic Setup
import os
from dotenv import load_dotenv, find_dotenv

# Load local env file
_ = load_dotenv(find_dotenv())

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", api_key=os.environ['OPENAI_API_KEY'])


#### Define Search Tool and its configuration

In [32]:
# Agent specific imports
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

#### Create Agent that can run action
- Use Langgraph

In [33]:
from langgraph.prebuilt import create_react_agent
# Define agent executor by providing the base model and search tool separately as input
# remember model with tools that is defined above is NOT used here
# react agent can bind the search toool
agent_executor = create_react_agent(model, [search])

#### Run the agent with example that does not require LLM to use Search Tool

In [ ]:
# Run the agent with example that does not require LLM to use Search Tool
from langchain_core.messages import HumanMessage
response = agent_executor.invoke({"messages": [HumanMessage(content="Hello")]})
print(f"response[messages] : {response["messages"]}")

#### Run the agent with example that requires LLM to use Search Tool

In [ ]:
# Run the agent with example that requires LLM to use Search Tool
response = agent_executor.invoke({"messages": [HumanMessage(content="What is the weather like in Sparks, Maryland today?")]})
print(f"response[messages] : {response["messages"]}")

#### Stream Responses instead of all at once print
- In case the agent has to execute multiple steps a better approach is to stream output

In [ ]:
segment = 1
for msg_chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the weather like in Sparks, Maryland today?")]}
):
    print("========>")
    print(f"message segment {segment} : {msg_chunk}")
    segment += 1

#### Add Memory to make the agent stateful
- Use langgraph memory : This checkpoint saver stores checkpoints in memory using a `defaultdict`

In [41]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
agent_executor = create_react_agent(model, [search], checkpointer=memory)


In [ ]:
# Test by invoking agent

# Configure a session id to track thread
config = {"configurable": {"thread_id": "utkal-11-03-24"}}
segment = 1
for msg_chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the weather like in Sparks, Maryland today?")]},
    config=config
):
    print("========>")
    print(f"message segment {segment} : {msg_chunk}")
    segment += 1

In [ ]:
# Test memory by asking a follow up question
for msg_chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="How is it going to be this whole week?")]},
    config=config

):
    print("========>")
    print(f"message segment {segment} : {msg_chunk}")
    segment += 1